In [36]:
%load_ext sql
%sql sqlite:///olap.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


u'Connected: None@olap.db'

<h1>OLAP and Cubes Activity</h1>

<h1>1 Data and Motivation</h1>
<p>You're given a bunch of data on search queries by users. (We can pretend that these users are Google search users and you are an engineer on the Google Web Search team). You want to analyze the number of search queries made, who they are made by, and how successful your search engine is at returning a result that people want to click on. A particular user can only make one query at a time.</p>
<p>Below is a table called raw_search_log containing details of search queries.</p>

<p>raw_search_log</p>
<table>
<tr>
    <th>Field</th>
    <th>Type</th>
    <th>Description</th>
</tr>
<tr>
    <td>user_id</td>
    <td>INTEGER</td>
    <td>ID of the user that made the search.</td>
</tr>
<tr>
    <td>timestamp</td>
    <td>TIMESTAMP</td>
    <td>Time at which the search occurred. A particular
user_id can only make a single query at a given
timestamp.</td>
</tr>
<tr>
    <td>query</td>
    <td>VARCHAR(500)</td>
    <td>Search query (text typed into the search bar).</td>
</tr>
<tr>
    <td>rank</td>
    <td>INTEGER</td>
    <td>The rank of the search result they clicked on (how
high it appears in the results). This is NULL if they
never clicked on a result.</td>
</tr>
<tr>
    <td>click_url</td>
    <td>VARCHAR(200)</td>
    <td>The URL of the search result they clicked on. This is
NULL if they never clicked on a result.</td>
</tr>
<tr>
    <td>city</td>
    <td>VARCHAR(50)</td>
    <td>The location of the user (is constant for a given user).</td>
</tr>
<tr>
    <td>age</td>
    <td>INTEGER</td>
    <td>The age of the user (is constant for a given user).</td>
</tr>
</table>

<h1>2 OLAP Database Design</h1>
<p>We will split up the attributes of the raw data schema into a star schema with 2 dimension tables
(users_dim, dates_dim), and a fact table (searches_fact). Design the star schema.</p>

<center>users_dim</center>
<table>
<tr>
    <td>user_id</td>
    <td>INTEGER</td>
</tr>
<tr>
    <td>city</td>
    <td>VARCHAR(50)</td>
</tr>
<tr>
    <td>age</td>
    <td>INTEGER</td>
</tr>
</table>
<center>User information.</center>
<br/>

<center>searches_fact</center>
<table>
<tr>
    <td>user_id</td>
    <td>INTEGER</td>
</tr>
<tr>
    <td>timestamp</td>
    <td>TIMESTAMP</td>
</tr>
<tr>
    <td>query</td>
    <td>VARCHAR(500)</td>
</tr>
<tr>
    <td>rank</td>
    <td>INTEGER</td>
</tr>
<tr>
    <td>click_url</td>
    <td>VARCHAR(200)</td>
</tr>
</table>
<center>Search queries that were performed
(who, when, what).</center>
<br/>

<center>dates_dim</center>
<table>
<tr>
    <td>timestamp</td>
    <td>TIMESTAMP</td>
</tr>
<tr>
    <td>date</td>
    <td>DATE</td>
</tr>
<tr>
    <td>hour</td>
    <td>INTEGER</td>
</tr>
<tr>
    <td>is_weekend</td>
    <td>BOOLEAN</td>
</tr>
</table>
<center>Date/time information on
search queries.</center>
<br/>

<p>
1. Write the CREATE TABLE statement for users_dim. Don't forget about the primary key!
</p>

In [37]:
%%sql
drop table if exists users_dim;
create table users_dim(
    user_id INTEGER primary key,
    city VARCHAR(50),
    age INTEGER
);

Done.
Done.


[]

<p>2. Write the CREATE TABLE statement for dates_dim. This table contains attributes not found in the raw schema because we want to be able to do more detailed analysis on the data. Don't forget about the primary key!</p>

In [38]:
%%sql
drop table if exists dates_dim;
create table dates_dim(
    timestamp TIMESTAMP primary key,
    date DATE,
    hour INTEGER,
    is_weekend BOOLEAN
)

Done.
Done.


[]

<p>3. Write the CREATE TABLE statement for searches_fact. Don't forget about foreign keys into the dimension tables and the primary key. (If you forgot the FOREIGN KEY syntax, take a look at Lecture 5: Design Theory3 from the course website).</p>

In [39]:
%%sql
drop table if exists searches_fact;
create table searches_fact(
    user_id INTEGER,
    timestamp TIMESTAMP,
    query VARCHAR(500),
    rank INTEGER,
    click_url VARCHAR(200),
    primary key (user_id, timestamp),
    foreign key (user_id) references users_dim(user_id),
    foreign key (timestamp) references dates_dim(timestamp)
)

Done.
Done.


[]

<p>Discussion Why did we design it this way? What kind of queries might we be able to do with this schema design?</p>

<h1>3 Populating Tables</h1>
<p>After designing the tables, we need to populate them with data from raw_search_log in order to do our analysis!</p>
<p>Example. Populate the users_dim table with an INSERT...SELECT statement. [18250 rows ]</p>

In [40]:
%%sql
INSERT INTO users_dim (user_id, city, age)
SELECT DISTINCT user_id, city, age
FROM raw_search_log;

18250 rows affected.


[]

<p>
1. Populate the dates_dim table with an INSERT...SELECT statement. [28138 rows ]
</p>
<ul>
<li>You must extract the date (using the DATE function on the timestamp field) and hour of day
(using the EXTRACT function) from timestamp. The documentation for EXTRACT is here4 and
the syntax is EXTRACT(<field> FROM timestamp).
</li>
<li>
You must figure out whether or not the date is a weekend. Look at the EXTRACT function
again. You will want to use a CASE statement.
</li>
</ul>

In [41]:
%%sql
INSERT INTO dates_dim (timestamp, date, hour, is_weekend)
SELECT DISTINCT timestamp, DATE(timestamp), strftime('%H', timestamp), strftime('%w', timestamp)=='0' or strftime('%w', timestamp)=='6'
FROM raw_search_log;

28138 rows affected.


[]

<p>
2. Write an INSERT...SELECT statement to populate the searches_fact table. [28195 rows ]
</p>

In [42]:
%%sql
INSERT INTO searches_fact (user_id, timestamp, query, rank, click_url)
SELECT DISTINCT user_id, timestamp, query, rank, click_url
FROM raw_search_log;

28195 rows affected.


[]

<p>Discussion Think about how much work was required to set up the data for analysis. Compare this to NoSQL/Redis. How much design work did you have to do before analyzing the data?</p>
<p>What if you wanted to add a field source to denote where the user queried from (desktop, Android browser, Safari, etc.)? What would you have to do in SQL versus what you do in Redis? What if source were unknown? How would this be denoted in SQL versus Redis?</p>

<h1>4 OLAP Queries</h1>
<p>Now, we want to use our dimension and fact tables to analyze the data. We don't want to use raw_search_log as that would defeat the purpose of this exercise. NATURAL JOINs will come in handy and are easy to use by design of the schema.</p>
<p>Example. Find the number of queries performed by people between the ages of 18 and 25, ordered by age.</p>


In [43]:
%%sql
SELECT age, COUNT(query) AS num_queries
FROM users_dim NATURAL JOIN searches_fact
WHERE age BETWEEN 18 and 25 GROUP BY age ORDER BY age;

Done.


age,num_queries
18,707
19,743
20,651
21,691
22,730
23,715
24,672
25,545


<h2>Problems</h2>
<p>1. How many characters long are the search queries done by people at Stanford? Find the top 10 longest queries and their length for queries done by users in Stanford. LENGTH (documentation here6) and LIMIT (documentation here7) will be helpful. [Longest query is 255 characters ]</p>

In [44]:
%%sql
select length(s.query) as query_len, s.query
from users_dim as u natural join searches_fact as s
where u.city = 'Stanford'
order by length(s.query) desc
limit 10

Done.


query_len,query
255,phone numbers netherlandso&client pub-7240984912124482&forid 1&ie iso-8859-1&oe iso-8859-1&cof galt 3a 23008000 3bgl 3a1 3bdiv 3a 23336699 3bvlc 3a663399 3bah 3acenter 3bbgc 3affffff 3blbgc 3a336699 3balc 3a0000ff 3blc 3a0000ff 3bt 3a000000 3bgfnt 3a0000f
97,action guidpresentlogin&guid nhw1mxwrbythak9lsxlgze5dcziytvh5dvnuwctrqniycujlyq &logincode retail
64,william and mary summer history program for high school students
63,listen tolil wayne fat joe paul wall rick ross holla at me baby
63,reviews for some people some other place by j california cooper
62,how do i delete 360 sharepro from my system after uninstalling
62,windows 98 cd burning tools for burning audio images and video
59,why do christian husbands fall out of love with thier wives
58,how long does it take to download music if the speed is t3
57,sermon of the 20th century because he had reached ...http


<p>2. We want to get summary statistics on queries in each city. For each city, get the number of unique users and number of queries made. The result's schema should be (city, num_users, num_queries).</p>

In [45]:
%%sql
select city, count(distinct u.user_id) as num_users, count(s.query) as num_queries
from users_dim as u natural join searches_fact as s
group by u.city

Done.


city,num_users,num_queries
Austin,892,1381
Baltimore,883,1414
Boston,841,1247
Boulder,873,1388
Cupertino,863,1311
Dallas,876,1382
Hillsdale,842,1340
Las Vegas,852,1321
Los Angeles,865,1257
New York,903,1434


<p>3. Find the maximum length of search queries each day (not timestamp!) that returned a click_url (i.e. searches that resulted in a user clicking on a search result). We're interested in seeing if the search engine is getting better at returning results for queries, or if people are better at searching with shorter queries. The result should be of the form (date, max_query_length).</p>

In [46]:
%%sql
select date, max(length(s.query)) as max_query_length
from searches_fact as s natural join dates_dim as d
where click_url is not NULL
group by d.date

Done.


date,max_query_length
2006-03-01,78
2006-03-02,76
2006-03-03,55
2006-03-04,41
2006-03-05,52
2006-03-06,52
2006-03-07,72
2006-03-08,48
2006-03-09,62
2006-03-10,61


<p>4. Find the bad kids! Write a query that returns the user_ids of users under the age of 18 who've made searches between the hours of 2 am and 7 am (hour values of 2 and 7), inclusive. [176 rows]
</p>

In [47]:
%%sql
select distinct u.user_id
from users_dim as u natural join searches_fact as s natural join dates_dim as d
where u.age < 18 and d.hour between 2 and 7

Done.


user_id
34760
118401
130832
134213
294499
323500
432273
436356
607103
629009


<h1>5 Discussion</h1>
<p>Think about these questions below and we will discuss it together as a class.</p>
<ul>
<li>Installation and setup-wise, what was easier to do in Redis?</li>
<li>What kind of analysis was easier to do in SQL?</li>
<li>What are some of the disadvantages or limitations of using SQL rather than Redis?</li>
</ul>

<h1>6 Bonus Problems (Extra Practice)</h1>
<p>Do these if you have extra time! This is not required as part of the activity.</p>
<p>1. Report number of queries and clicked websites viewed by each user in Palo Alto. The result should be of the form (user_id, num_queried, num_clicked). [First row of results is (281371, 2, 2) ]</p>

In [48]:
%%sql
select user_id, count(s.query) as num_queried, count(s.click_url) as num_clicked
from users_dim as u natural join searches_fact as s
where u.city = 'Palo Alto'
group by u.user_id 

Done.


user_id,num_queried,num_clicked
13379,1,0
46669,1,0
54537,1,1
58965,2,1
60847,1,1
72710,8,6
73450,4,2
91602,28,18
126937,4,4
135796,1,1


<p>2. For dates between '2006-03-04' and '2006-03-07', find the number of queries. Your result should be of the form (date, num_queries). Remember, some queries could have occurred at the same time! [335, 400, 348, 369]</p>

In [49]:
%%sql
select date, count(s.query) as num_queries
from searches_fact as s natural join dates_dim as d
where d.date between '2006-03-04' and '2006-03-07'
group by d.date

Done.


date,num_queries
2006-03-04,335
2006-03-05,400
2006-03-06,348
2006-03-07,369


<p>3. Are older people less effective at making a good search? Get the number of queries that did not return a click_url from users 50 or older who don't live in Oldsville. [2430 ]</p>

In [50]:
%%sql
select count(u.user_id) as num_queries
from users_dim as u natural join searches_fact as s
where u.age >= 50 and u.city <> 'Oldsville' and s.click_url is NULL;

Done.


num_queries
2430


<p>4. Super-Bonus! It would be interesting to see if the average queries per hour is different between days during the week and days during the weekend. Do people make more searches during the weekend? Do they make more at night? Write a query that computes the number of queries per hour, averaged over the weekdays and over the weekends. The result should have rows and columns like this:</p>

<table>
<tr>
    <th>hour</th>
    <th>avg_weekday_queries</th>
    <th>avg_weekend_queries</th>
</tr>
<tr>
    <td>0</td>
    <td>11.89</td>
    <td>10.69</td>
</tr>
<tr>
    <td>1</td>
    <td>7.58</td>
    <td>9.27</td>
</tr>
<tr>
    <td>...</td>
    <td>...</td>
    <td>...</td>
</tr>
</table>

<p>avg_weekday_queries contains the number of queries per hour, averaged over all weekdays (the is_weekend field in dates_dim will be helpful), and avg_weekend_queries contains the number of queries per hour, averaged over all weekend days.</p>
<p>Hint: Break it apart into two queries (one for the weekend, one for the weekday) and join them together at the end.
</p>

In [51]:
%%sql
select tab1.hour, avg_weekday_queries, avg_weekend_queries
from
    (select d.hour, count(s.query)/5.0 as avg_weekday_queries
     from searches_fact as s natural join dates_dim as d
     where d.is_weekend = 0
     group by d.hour) as tab1,
    (select d.hour, count(s.query)/2.0 as avg_weekend_queries
     from searches_fact as s natural join dates_dim as d
     where d.is_weekend = 1
     group by d.hour) as tab2
where tab1.hour = tab2.hour

Done.


hour,avg_weekday_queries,avg_weekend_queries
0,157.0,139.0
1,98.6,120.5
2,52.4,59.5
3,40.4,43.0
4,30.8,31.0
5,31.4,24.0
6,39.8,43.0
7,63.8,65.5
8,104.8,108.5
9,141.6,161.5
